In [1]:
import os

PATH = '/home/stadlerm/Development/Projects/DreamChallenge/RA2_alpine_lads/ra_joint_predictions'

os.chdir(PATH)

In [2]:
from utils.config import Config

config = Config()

In [3]:
import os
import pandas as pd

training_csv = os.path.join(config.train_location, 'training.csv')

training_data = pd.read_csv(training_csv)

In [4]:
import re

import numpy as np

# Prepare outcome vector
column_types = [
    ['hands_narrowing', '.*H_.*_J.*'], 
    ['hands_erosion', '.*H_.*_E.*'], 
    ['feet_narrowing', '.*F_.*_J.*'],
    ['feet_erosion', '.*F_.*_E.*']
]

for column_type in column_types:
    file_post_fix = column_type[0]
    column_reg = column_type[1]
    
    p = re.compile(column_reg)

    column_idx = np.where([p.match(column) for column in training_data.columns])[0]
    columns = training_data.columns[column_idx]
    
    values = training_data[columns]
    vals, counts = np.unique(values, return_counts = True)
    
    y = np.zeros((training_data.shape[0], vals.size))
    
    for idx, val in enumerate(vals):
        r_idx = np.where(values == val)[0]
        
        y[r_idx, idx] = 1
        
    y_t = y[:, 1]

    for idx in range(1, vals.size):
        y_t = np.logical_and(y_t, y[:, idx])
    
    print(np.count_nonzero(y_t))

6
2
1
0


In [5]:
import functools

y_t = y[:, 0]

for idx in range(1, 5):
    y_t = np.logical_and(y_t, y[:, idx])
    
print(np.count_nonzero(y_t))

4


In [6]:
# Regex for hand narrowing, erosion, and feet narrowing columns
column_reg = ['.*F_.*_J.*']

for idx, reg in enumerate(column_reg):
    p = re.compile(reg)

    column_idx = np.where([p.match(column) for column in training_data.columns])[0]
    columns = training_data.columns[column_idx]
    
    values = training_data[columns].to_numpy()
    vals, c = np.unique(values, return_counts = True)
    
    sorted_idx = np.argsort(c)[::-1]
    
    y = np.zeros(training_data.shape[0])
    for idx in sorted_idx[1:]:
        y[np.where(values == vals[idx])[0]] = vals[idx]

In [7]:
print(np.unique(y, return_counts = True))

(array([0., 1., 2., 3., 4.]), array([257,   7,  47,  28,  29]))


In [8]:
from sklearn.model_selection import train_test_split

train_idx, test_idx = train_test_split(np.arange(training_data.shape[0]), test_size = 0.25, stratify = y)

In [9]:
print(np.unique(y[train_idx], return_counts = True))

(array([0., 1., 2., 3., 4.]), array([193,   5,  35,  21,  22]))


In [10]:
print(np.unique(y[test_idx], return_counts = True))

(array([0., 1., 2., 3., 4.]), array([64,  2, 12,  7,  7]))


In [11]:
train_patient_id = training_data.iloc[train_idx]['Patient_ID'].to_numpy()
test_patient_id = training_data.iloc[test_idx]['Patient_ID'].to_numpy()

In [12]:
print(train_patient_id)

['UAB648' 'UAB628' 'UAB271' 'UAB128' 'UAB562' 'UAB111' 'UAB099' 'UAB236'
 'UAB068' 'UAB592' 'UAB101' 'UAB492' 'UAB176' 'UAB310' 'UAB595' 'UAB531'
 'UAB246' 'UAB352' 'UAB395' 'UAB067' 'UAB051' 'UAB106' 'UAB482' 'UAB040'
 'UAB078' 'UAB115' 'UAB617' 'UAB297' 'UAB331' 'UAB524' 'UAB486' 'UAB719'
 'UAB392' 'UAB079' 'UAB189' 'UAB270' 'UAB342' 'UAB039' 'UAB024' 'UAB703'
 'UAB070' 'UAB086' 'UAB563' 'UAB278' 'UAB536' 'UAB179' 'UAB708' 'UAB567'
 'UAB009' 'UAB216' 'UAB109' 'UAB477' 'UAB228' 'UAB665' 'UAB333' 'UAB139'
 'UAB500' 'UAB666' 'UAB233' 'UAB519' 'UAB022' 'UAB035' 'UAB061' 'UAB323'
 'UAB497' 'UAB630' 'UAB475' 'UAB093' 'UAB007' 'UAB223' 'UAB625' 'UAB037'
 'UAB427' 'UAB219' 'UAB084' 'UAB711' 'UAB503' 'UAB138' 'UAB268' 'UAB208'
 'UAB057' 'UAB098' 'UAB451' 'UAB539' 'UAB399' 'UAB126' 'UAB317' 'UAB052'
 'UAB661' 'UAB692' 'UAB107' 'UAB191' 'UAB113' 'UAB087' 'UAB461' 'UAB443'
 'UAB487' 'UAB105' 'UAB124' 'UAB182' 'UAB150' 'UAB159' 'UAB224' 'UAB481'
 'UAB413' 'UAB464' 'UAB135' 'UAB272' 'UAB580' 'UAB1

In [13]:
print(test_patient_id)

['UAB557' 'UAB162' 'UAB574' 'UAB394' 'UAB506' 'UAB521' 'UAB302' 'UAB488'
 'UAB050' 'UAB232' 'UAB654' 'UAB326' 'UAB123' 'UAB601' 'UAB504' 'UAB420'
 'UAB129' 'UAB678' 'UAB366' 'UAB196' 'UAB142' 'UAB639' 'UAB060' 'UAB626'
 'UAB264' 'UAB301' 'UAB343' 'UAB114' 'UAB056' 'UAB217' 'UAB090' 'UAB063'
 'UAB707' 'UAB286' 'UAB700' 'UAB177' 'UAB231' 'UAB569' 'UAB197' 'UAB620'
 'UAB130' 'UAB490' 'UAB036' 'UAB300' 'UAB088' 'UAB566' 'UAB517' 'UAB680'
 'UAB207' 'UAB318' 'UAB403' 'UAB405' 'UAB163' 'UAB596' 'UAB049' 'UAB319'
 'UAB345' 'UAB716' 'UAB145' 'UAB499' 'UAB572' 'UAB096' 'UAB363' 'UAB463'
 'UAB280' 'UAB706' 'UAB041' 'UAB640' 'UAB046' 'UAB577' 'UAB379' 'UAB151'
 'UAB032' 'UAB002' 'UAB030' 'UAB018' 'UAB172' 'UAB065' 'UAB511' 'UAB675'
 'UAB667' 'UAB180' 'UAB108' 'UAB075' 'UAB637' 'UAB452' 'UAB398' 'UAB384'
 'UAB389' 'UAB472' 'UAB422' 'UAB701']


In [14]:
print(np.intersect1d(train_patient_id, test_patient_id))

[]


In [15]:
hands_dataframe = pd.read_csv('./data/predictions/hand_joint_data.csv')
feet_dataframe = pd.read_csv('./data/predictions/feet_joint_data.csv')

In [16]:
hand_idx = [image_name.split('-')[0] in test_patient_id for image_name in hands_dataframe['image_name']]
feet_idx = [image_name.split('-')[0] in test_patient_id for image_name in feet_dataframe['image_name']]

In [17]:
hands_dataframe.iloc[np.where(np.logical_not(hand_idx))[0]].to_csv('./data/predictions/hand_joint_data_train.csv')
feet_dataframe.iloc[np.where(np.logical_not(feet_idx))[0]].to_csv('./data/predictions/feet_joint_data_train.csv')

hands_dataframe.iloc[np.where(hand_idx)[0]].to_csv('./data/predictions/hand_joint_data_test.csv')
feet_dataframe.iloc[np.where(feet_idx)[0]].to_csv('./data/predictions/feet_joint_data_test.csv')

In [18]:
feet_data = pd.read_csv('./data/predictions/feet_joint_data_train.csv')
feet_data_val = pd.read_csv('./data/predictions/feet_joint_data_test.csv')